In [3]:
import json
import sys
import math
from tqdm import tqdm
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import copy
from sklearn import tree
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingClassifier
from sklearn import svm
import pickle

## LDA_100

In [24]:
train_path = 'train_data/all_train_100.csv'
train_data = pd.read_csv(train_path)
train_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,2.5,1.0,0.0,0.0,1.0,1.0,0.0,...,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204
1,108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,3.5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189
2,300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,4.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526
3,318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,3.5,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.039897,0.000385
4,331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,3.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.001111,0.001111,0.223333,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111


In [25]:
test_path = 'train_data/all_test_100.csv'
test_data = pd.read_csv(test_path)
test_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
0,108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,3.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.000012,0.000027,0.000005,9.320489e-07,0.000043,9.320489e-07,0.000002,0.000013,0.000032,0.000012
1,499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,4.5,1.0,1.0,1.0,1.0,0.0,0.0,...,0.098289,0.000005,0.000791,4.591333e-06,0.000017,4.591333e-06,0.001446,0.000005,0.000943,0.000019
2,533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,4.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.027581,0.000005,0.000386,4.700108e-06,0.000059,4.700108e-06,0.000846,0.000042,0.001169,0.000064
3,190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.001919,0.000024,0.000013,5.558423e-06,0.000086,5.577801e-06,0.000010,0.000086,0.000042,0.001630
4,394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.001160,0.000020,0.000016,4.482868e-06,0.000015,4.482868e-06,0.000009,0.000050,0.000113,0.001357


In [26]:
train_x = train_data.dropna()
train_y = np.array(train_x['Ratings']).astype(float).ravel()
del train_x['Ratings']
del train_x['business_id']
del train_x['user_id']
del train_x['Unnamed: 0']
test_x = test_data.dropna()
test_y = np.array(test_x['Ratings']).astype(float).ravel()
del test_x['Ratings']
del test_x['business_id']
del test_x['user_id']
del test_x['Unnamed: 0']

In [28]:
# regression methods

def get_round_number(x):
    if x > 5:
        return 5
    if x < 0.5:
        return 0.5
    a = int(x)
    b = x - a
    if b < 0.25:
        return float(a)
    elif b > 0.75:
        return float(a+1)
    else:
        return a + 0.5
    
slr = LinearRegression()
slr.fit(train_x, train_y)
y = slr.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
filename = 'model/LDA100_linear.pickle'
pickle.dump(slr, open(filename, 'wb'))
print('Linear Regression: ', np.sqrt(mean_squared_error(y_pred, test_y)))

clf = AdaBoostRegressor(base_estimator=None, n_estimators=50,
                            learning_rate=1.0, loss='exponential',
                            random_state=None)
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('AdaBoost', np.sqrt(mean_squared_error(y_pred, test_y)))

Linear Regression:  0.637117773442346
AdaBoost 0.7160460568971959


In [31]:
train_y = (train_y*2).astype(float).ravel()
test_y = (test_y*2).astype(float).ravel()

In [45]:
# classification methods

def get_round_number(x):
    if x > 10:
        return 10
    if x < 1:
        return 1
    a = int(x)
    b = x - a
    if b <= 0.5:
        return float(a)
    elif b > 0.5:
        return float(a+1)

clf = LogisticRegression(penalty='l2')
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('Logistic Regression: ', np.sqrt(mean_squared_error(np.array(y_pred)/2, test_y/2)))


clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, min_samples_split=.01,
                                 n_jobs=4, random_state=None, verbose=0)
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('RandomForest', np.sqrt(mean_squared_error(np.array(y_pred)/2, test_y/2)))

model = tree.DecisionTreeClassifier(criterion = 'entropy')
model = model.fit(train_x, train_y)
y_pred = model.predict(test_x)
print('Decision Tree: ', np.sqrt(mean_squared_error(y_pred/2, test_y/2)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression:  0.7023072451343648
RandomForest 0.7089253525233482
Decision Tree:  0.7827819957450034


## LDA_50

In [12]:
train_path = 'train_data/all_train_50.csv'
train_data = pd.read_csv(train_path)
train_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50
0,108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,2.5,1.0,0.0,0.0,1.0,1.0,0.0,...,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.000408,0.020571,0.083094
1,108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,3.5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.000377,0.000377,0.000377,0.000377,0.000377,0.020893,0.000377,0.000377,0.000377,0.000377
2,300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,4.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053,0.001053
3,318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,3.5,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000769,0.000769,0.039231,0.000769,0.000769,0.000769,0.000769,0.000769,0.000769,0.132505
4,331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,3.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222,0.002222


In [13]:
test_path = 'train_data/all_test_50.csv'
test_data = pd.read_csv(test_path)
test_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,topic_41,topic_42,topic_43,topic_44,topic_45,topic_46,topic_47,topic_48,topic_49,topic_50
0,108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,3.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.000390,0.000017,0.000015,0.000003,0.000037,0.000368,0.000116,0.000017,0.000730,0.004353
1,499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,4.5,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000088,0.000012,0.001105,0.000007,0.000039,0.000023,0.000298,0.000018,0.000031,0.003686
2,533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,4.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000289,0.000038,0.001584,0.000008,0.000052,0.000015,0.000358,0.000034,0.000008,0.004278
3,190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.000185,0.006823,0.000040,0.000013,0.000085,0.001181,0.001187,0.000427,0.000086,0.000065
4,394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.000247,0.001771,0.000419,0.000017,0.000063,0.001751,0.001892,0.000312,0.000091,0.000070


In [14]:
train_x = train_data.dropna()
train_y = np.array(train_x['Ratings']).astype(float).ravel()
del train_x['Ratings']
del train_x['business_id']
del train_x['user_id']
del train_x['Unnamed: 0']
test_x = test_data.dropna()
test_y = np.array(test_x['Ratings']).astype(float).ravel()
del test_x['Ratings']
del test_x['business_id']
del test_x['user_id']
del test_x['Unnamed: 0']

In [15]:
# regression methods

def get_round_number(x):
    if x > 5:
        return 5
    if x < 0.5:
        return 0.5
    a = int(x)
    b = x - a
    if b < 0.25:
        return float(a)
    elif b > 0.75:
        return float(a+1)
    else:
        return a + 0.5
    
slr = LinearRegression()
slr.fit(train_x, train_y)
y = slr.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
filename = 'model/LDA100_linear.pickle'
pickle.dump(slr, open(filename, 'wb'))
print('Linear Regression: ', np.sqrt(mean_squared_error(y_pred, test_y)))

clf = AdaBoostRegressor(base_estimator=None, n_estimators=50,
                            learning_rate=1.0, loss='exponential',
                            random_state=None)
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('AdaBoost', np.sqrt(mean_squared_error(y_pred, test_y)))

Linear Regression:  0.6349297521274252
AdaBoost 0.7290125120262045


In [16]:
train_y = (train_y*2).astype(float).ravel()
test_y = (test_y*2).astype(float).ravel()

In [17]:
# classification methods

def get_round_number(x):
    if x > 10:
        return 10
    if x < 1:
        return 1
    a = int(x)
    b = x - a
    if b <= 0.5:
        return float(a)
    elif b > 0.5:
        return float(a+1)

clf = LogisticRegression(penalty='l2')
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('Logistic Regression: ', np.sqrt(mean_squared_error(np.array(y_pred)/2, test_y/2)))


clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, min_samples_split=.01,
                                 n_jobs=4, random_state=None, verbose=0)
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('RandomForest', np.sqrt(mean_squared_error(np.array(y_pred)/2, test_y/2)))

model = tree.DecisionTreeClassifier(criterion = 'entropy')
model = model.fit(train_x, train_y)
y_pred = model.predict(test_x)
print('Decision Tree: ', np.sqrt(mean_squared_error(y_pred/2, test_y/2)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression:  0.699773599959268
RandomForest 0.6973566185845002
Decision Tree:  0.8211212584207624


## TF-IDF

In [4]:
train_path = 'train_data/tfidf_train.csv'
train_data = pd.read_csv(train_path)
train_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
0,108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,2.5,1.0,0.0,0.0,1.0,1.0,0.0,...,0.148474,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,3.5,1.0,0.0,0.0,0.0,0.0,1.0,...,0.142877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,4.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,3.5,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.294119
4,331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,3.5,0.0,0.0,0.0,0.0,0.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [5]:
test_path = 'train_data/tfidf_test.csv'
test_data = pd.read_csv(test_path)
test_data.head()

,Unnamed: 0,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,...,went,wife,wine,wish,wo,wonder,work,worth,wrong,year
0,108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,3.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.145676,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
1,499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,4.5,1.0,1.0,1.0,1.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.073530
2,533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,4.5,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.073530
3,190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,1.0,1.0,0.0,...,0.000000,0.0,0.0,0.077334,0.0,0.0,0.059389,0.259429,0.0,0.066885
4,394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.077334,0.0,0.0,0.059389,0.259429,0.0,0.066885


In [6]:
train_x = train_data.dropna()
train_y = np.array(train_x['Ratings']).astype(float).ravel()
del train_x['Ratings']
del train_x['business_id']
del train_x['user_id']
del train_x['Unnamed: 0']
test_x = test_data.dropna()
test_y = np.array(test_x['Ratings']).astype(float).ravel()
del test_x['Ratings']
del test_x['business_id']
del test_x['user_id']
del test_x['Unnamed: 0']

In [7]:
# regression methods

def get_round_number(x):
    if x > 5:
        return 5
    if x < 0.5:
        return 0.5
    a = int(x)
    b = x - a
    if b < 0.25:
        return float(a)
    elif b > 0.75:
        return float(a+1)
    else:
        return a + 0.5
    
slr = LinearRegression()
slr.fit(train_x, train_y)
y = slr.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
filename = 'model/tfidf_linear.pickle'
pickle.dump(slr, open(filename, 'wb'))
print('Linear Regression: ', np.sqrt(mean_squared_error(y_pred, test_y)))

clf = AdaBoostRegressor(base_estimator=None, n_estimators=50,
                            learning_rate=1.0, loss='exponential',
                            random_state=None)
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('AdaBoost', np.sqrt(mean_squared_error(y_pred, test_y)))

Linear Regression:  0.6780317452043506
AdaBoost 0.7747346624091995


In [8]:
train_y = (train_y*2).astype(float).ravel()
test_y = (test_y*2).astype(float).ravel()

In [11]:
# classification methods

def get_round_number(x):
    if x > 10:
        return 10
    if x < 1:
        return 1
    a = int(x)
    b = x - a
    if b <= 0.5:
        return float(a)
    elif b > 0.5:
        return float(a+1)

clf = LogisticRegression(penalty='l2')
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('Logistic Regression: ', np.sqrt(mean_squared_error(np.array(y_pred)/2, test_y/2)))


clf = RandomForestClassifier(n_estimators=50, criterion='gini', max_depth=None, min_samples_split=.01,
                                 n_jobs=4, random_state=None, verbose=0)
clf.fit(train_x, train_y)
y = clf.predict(test_x)
y_pred = []
for item in y:
    y_pred.append(get_round_number(item))
print('RandomForest', np.sqrt(mean_squared_error(np.array(y_pred)/2, test_y/2)))

model = tree.DecisionTreeClassifier(criterion = 'entropy')
model = model.fit(train_x, train_y)
y_pred = model.predict(test_x)
print('Decision Tree: ', np.sqrt(mean_squared_error(y_pred/2, test_y/2)))

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Logistic Regression:  0.7457163383603936
RandomForest 0.7264045816246242
Decision Tree:  0.831837336661979
